# Function Calling Tutorial PT 4A

Okay in this example I'm going to use a query in which the question would need successive non parallel function calls.

1. Call money in bank account (in USD)
2. Convert that money to AUD

The tool calls are reliant on each other!

This tutorial shows our current structure is limited.

In [31]:
from openai import OpenAI
import json
import os
from dotenv import load_dotenv
import requests

load_dotenv()

client = OpenAI(api_key=os.getenv("OPENAI_API_KEY"))

In [32]:
!pip install pytz

In [33]:
def get_balance(account_type: str) -> str:
    # Mock response (pretend we called an API)
    balances = {
        "savings": "1000 USD",
        "checking": "500 USD",
        "investment": "2500 USD"
    }
    return balances.get(account_type.lower(), "Account not found")

# Example call
print(get_balance("savings"))


1000 USD


In [34]:
def convert_currency(base_currency, target_currency, amount):
    """
    Converts a specified amount from one currency to another using the ExchangeRate-API.
    """
    api_key = os.getenv("EXCHANGE_RATE_API")
    url = f"https://v6.exchangerate-api.com/v6/{api_key}/pair/{base_currency}/{target_currency}"
    
    response = requests.get(url)
    
    if response.status_code == 200:
        data = response.json()
        if "conversion_rate" in data:
            converted_amount = amount * data["conversion_rate"]
            return f"{converted_amount} {target_currency}"
        else:
            return "Invalid currency pair or data unavailable"
    else:
        return f"Error: {response.status_code}, {response.text}"

print(convert_currency("USD", "EUR", 100))

85.03 EUR


In [35]:
import pytz
from datetime import datetime

def get_current_time(timezone):
    """
    Fetches the current time in the specified timezone.
    """
    try:
        tz = pytz.timezone(timezone)
        current_time = datetime.now(tz).strftime("%Y-%m-%d %H:%M:%S %Z")
        return current_time
    except pytz.UnknownTimeZoneError:
        return "Invalid timezone. Please provide a valid IANA timezone."

print(get_current_time("Australia/Sydney"))

2025-09-16 23:00:26 AEST


In [36]:
# Define the function you are going to use:
# Uses Open-Meteo's Weather Forecast API!!!
def get_weather(latitude, longitude):
    url = (
        f"https://api.open-meteo.com/v1/forecast?"
        f"latitude={latitude}&longitude={longitude}"
        f"&current=temperature_2m"
    )
    
    response = requests.get(url)
    data = response.json()
    
    return data.get('current').get('temperature_2m')

# example use
get_weather("-33.8727", "151.2057")

19.8

In [ ]:
# define list of callable tools for model
tools = [
    {
        "type": "function",
        "name": "get_current_weather",
        "description": "Get the current weather in degrees Celcius for a given latitude and longitude",
        "parameters": {
            "type": "object",
            "properties": {
                "latitude": {
                    "type": "string",
                    "description": "The latitude of a place",
                },
                "longitude": {
                    "type": "string",
                    "description": "The longitude of a place",
                },
            },
            "required": ["latitude", "longitude"],
        }
    },
    {
        "type": "function",
        "name": "get_current_time",
        "description": "Get the current time for given IANA timezone",
        "parameters": {
            "type": "object",
            "properties": {
                "IANA_timezone": {
                    "type": "string",
                    "description": "The IANA timezone of a place",
                },
            },
            "required": ["IANA_timezone"]
        }
    },
    {
        "type": "function",
        "name": "convert_currency",
        "description": "Converts a specified amount from one currency to another.",
        "parameters": {
            "type": "object",
            "properties": {
                "base_currency": {
                    "type": "string", 
                    "description": "Base currency code (e.g., 'USD')."
                },
                "target_currency": {
                    "type": "string",
                    "description": "Target currency code (e.g., 'AUD')."
                },
                "amount": {
                    "type": "number",
                    "description": "Amount to be converted."
                }
            },
            "required": ["base_currency", "target_currency", "amount"],
        },
    }, 
    {
        "type": "function",
        "name": "get_balance",
        "description": "Gets the balance in my bank account from either my savings, checking or investment account",
        "parameters": {
            "type": "object",
            "properties": {
                "account_type": {
                    "type": "string", 
                    "description": "Either 'savings', 'checking' or 'investment' account"
                },
            },
            "required": ["account_type"],
        },
    }
]

messages = [{"role": "user", "content": "How much money in my savings account converted into AUD?"}]
# ERROR! Current setup doesn't work unless you change the format so that 
# tool calls aren't all just happening in parallel. You need to loop.

response = client.responses.create(
    model="gpt-3.5-turbo",
    tools=tools,
    input=messages,
    tool_choice="auto",
)

#print(response.model_dump_json(indent=2))
print(response.output)

# WHAT HAPPENED?
# We called the api endpoint and it returned which tool we should use and the inputs

[ResponseFunctionToolCall(arguments='{"account_type":"savings"}', call_id='call_qqA5zzgdW1Un5GRpi4rP9KLx', name='get_balance', type='function_call', id='fc_0e7dccc68c8844520068c95f719cc0819ea5e32de9c410ca18', status='completed')]


In [ ]:
function_call = None
function_call_arguments = None
messages += response.output

for item in response.output:
    if item.type == "function_call":
        function_call = item
        function_call_arguments = json.loads(item.arguments)

        print(function_call)
        print(function_call_arguments)

        # 3. Execute the function logic for each function:
        if function_call.name == "get_current_weather":
            result = {"weather": get_weather(function_call_arguments["latitude"], 
                                             function_call_arguments["longitude"])}

        if function_call.name == "get_current_time":
            result = {"time": get_current_time(function_call_arguments["IANA_timezone"])}
        
        if function_call.name == "convert_currency":
            result = {"converted currency": convert_currency(function_call_arguments["base_currency"],
                                                             function_call_arguments["target_currency"],
                                                             function_call_arguments["amount"])}

        if function_call.name == "get_balance":
            result = {"bank account balance": get_balance(function_call_arguments["account_type"])}

        print(result)
        
        # 4. Provide function call results to the model
        messages.append({
            "type": "function_call_output",
            "call_id": function_call.call_id,
            "output": json.dumps(result),
        })

        from pprint import pprint
        pprint(messages)

ResponseFunctionToolCall(arguments='{"account_type":"savings"}', call_id='call_qqA5zzgdW1Un5GRpi4rP9KLx', name='get_balance', type='function_call', id='fc_0e7dccc68c8844520068c95f719cc0819ea5e32de9c410ca18', status='completed')
{'account_type': 'savings'}
{'bank account balance': '1000 USD'}
[{'content': 'How much money in my savings account converted into AUD?',
  'role': 'user'},
 ResponseFunctionToolCall(arguments='{"account_type":"savings"}', call_id='call_qqA5zzgdW1Un5GRpi4rP9KLx', name='get_balance', type='function_call', id='fc_0e7dccc68c8844520068c95f719cc0819ea5e32de9c410ca18', status='completed'),
 {'call_id': 'call_qqA5zzgdW1Un5GRpi4rP9KLx',
  'output': '{"bank account balance": "1000 USD"}',
  'type': 'function_call_output'}]


In [39]:
response = client.responses.create(
    model="gpt-3.5-turbo",
    tools=tools,
    input=messages,
)

# 5. The model should be able to give a response!
print("Final output:")
print(response.output_text + "\n\n")
print(response.model_dump_json(indent=2))


Final output:



{
  "id": "resp_0e7dccc68c8844520068c95f7234a0819e911bb2028f8acfdf",
  "created_at": 1758027634.0,
  "error": null,
  "incomplete_details": null,
  "instructions": null,
  "metadata": {},
  "model": "gpt-3.5-turbo-0125",
  "object": "response",
  "output": [
    {
      "arguments": "{\"base_currency\":\"USD\",\"target_currency\":\"AUD\",\"amount\":1000}",
      "call_id": "call_QtMJkoh6Vg6U8rSRTHmCW9jY",
      "name": "convert_currency",
      "type": "function_call",
      "id": "fc_0e7dccc68c8844520068c95f7344a8819eb1437b425f280e4e",
      "status": "completed"
    }
  ],
  "parallel_tool_calls": true,
  "temperature": 1.0,
  "tool_choice": "auto",
  "tools": [
    {
      "name": "get_current_weather",
      "parameters": {
        "type": "object",
        "properties": {
          "latitude": {
            "type": "string",
            "description": "The latitude of a place"
          },
          "longitude": {
            "type": "string",
            "descrip